In [19]:
# imports 

import re
import sqlite3
import emoji
import math

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
from datetime import datetime 

tqdm.pandas()

In [3]:
# paths 

path_to_db = '/Users/alexraudvee/Downloads/test.db'

path_to_conv = '/Users/alexraudvee/Desktop/TU:e/Q4Y1_data_challenge_1/conversations.json'

In [4]:
# connection to the database with sentiment scores

try:
    connection = sqlite3.connect(path_to_db)
    print('Succesfuly connected to the database')

    query = """
        SELECT name FROM sqlite_master  
        WHERE type='table';
        """

    cursor = connection.cursor()
    cursor.execute(query)
    list_of_tables = cursor.fetchall()

    print('---TABLE NAMES IN DB---')
    for table_name in list_of_tables:
        print('-----------------------')
        print(table_name[0])

    print('-----------------------')
    
except sqlite3.Error as e:
    print(e)


Succesfuly connected to the database
---TABLE NAMES IN DB---
-----------------------
users
-----------------------
tweets
-----------------------
tweets_geo
-----------------------
replies
-----------------------
retweets
-----------------------
quotes
-----------------------
hashtags
-----------------------
sqlite_sequence
-----------------------
symbols
-----------------------
user_mentions
-----------------------


In [5]:
# tweets table
def get_tweets(conn):
    query = """
        SELECT *
        FROM tweets
    """

    df = pd.read_sql(query, conn)
    return df

# users table
def get_users(conn):
    query = """
        SELECT *
        FROM users
    """

    df = pd.read_sql(query, conn)
    return df

# tweets geo table 
def get_tweets_geo(conn):
    query = """
        SELECT *
        FROM tweets_geo
    """

    df = pd.read_sql(query, conn)
    return df

# replies table
def get_replies(conn):
    query = """
        SELECT *
        FROM replies
    """

    df = pd.read_sql(query, conn)
    return df

# retweets table
def get_retweets(conn):
    query = """
        SELECT *
        FROM retweets
    """

    df = pd.read_sql(query, conn)
    return df

# quotes table
def get_quotes(conn):
    query = """
        SELECT *
        FROM quotes
    """

    df = pd.read_sql(query, conn)
    return df

# hastags table 
def get_hashtags(conn):
    query = """
        SELECT *
        FROM hashtags
    """

    df = pd.read_sql(query, conn)
    return df

# symbols table 
def get_symbols(conn):
    query = """
        SELECT *
        FROM symbols
    """

    df = pd.read_sql(query, conn)
    return df

# user mentions table 
def get_user_mentions(conn):
    query = """
        SELECT *
        FROM user_mentions
    """

    df = pd.read_sql(query, conn)
    return df

In [6]:
df_users = get_users(conn=connection)

In [7]:
df_users

,user_id,name,screen_name,location,followers_count,friends_count
0,22,rabble,rabble,PDX,16404,3182
1,155,Craig Calef,craigcalef,"BOS, SAN, SFO, LAS",2436,395
2,224,Dave Pell,davepell,San Francisco,54308,787
3,246,Blaine Cook,blaine,"London, UK",14968,2142
4,295,Josh Kopelman,joshk,Philly,132552,3795
...,...,...,...,...,...,...
2329595,1244677457028026370,Georgia,Georgia28337366,None,0,1
2329596,1244679780131311618,Flo,Flo43253301,None,0,0
2329597,1244685591809163264,Annie Beth,AnnieBe26893938,None,0,7
2329598,1244686973559803905,moons,M00nUn1t,None,0,0


In [27]:
# construct the dataframes for further analysis 

df_tweets = get_tweets(conn=connection)  
# df_users = get_users(conn=connection)
# df_replies = get_replies(conn=connection)
# df_retweets = get_retweets(conn=connection)
# df_quotes = get_quotes(conn=connection)
df_tweets_geo = get_tweets_geo(conn=connection)
# df_user_metions = get_user_mentions(conn=connection)
# df_symbols = get_symbols(conn=connection)
# df_hastags = get_hashtags(conn=connection)

In [28]:
# function to convert the sentiment label in numeric value

def convert_lable(lable):
    
    if lable == 'neutral':
        return int(0)
    elif lable == 'negative':
        return int(-1)
    elif lable == 'positive':
        return int(1)
    else:
        return None

In [29]:
# add new column with numeric values

df_tweets['sentiment_numeric'] = df_tweets['sentiment_label'].apply(lambda row: convert_lable(row))

In [30]:
df_tweets_geo = df_tweets_geo.rename(columns={'tweet_geo_id':'tweet_id'})

In [31]:
# construct the dataframe with conversations 

df_covs = pd.read_json(path_to_conv)

In [32]:
df_sentiment = df_tweets[['tweet_id', 'sentiment_numeric']] 

df_covs_sentiment = pd.merge(df_covs, df_sentiment, on='tweet_id')
df_covs_sentiment = pd.merge(df_covs_sentiment, df_tweets_geo, on='tweet_id')

In [33]:
df_covs_sentiment['respond_time'] = df_covs_sentiment['timestamp'].diff()

In [34]:
def convert_timedelta(datetime_obj):
   
    result = round((datetime_obj.seconds / 60))
                
    return result

In [35]:
df_covs_sentiment['respond_time'] = df_covs_sentiment['respond_time'].dropna().progress_apply(lambda row: convert_timedelta(row))

df_covs_sentiment

  0%|          | 1/40961 [00:00<4:25:40,  2.57it/s]


ValueError: cannot convert float NaN to integer